In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install colabcode
!pip install fastapi
!pip install pyngrok
!pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 1.8 MB/s 
     |████████████████████████████████| 8.3 MB 8.3 MB/s 
     |████████████████████████████████| 745 kB 38.9 MB/s 
     |████████████████████████████████| 344 kB 53.8 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 428 kB 44.7 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
     |████████████████████████████████| 132 kB 41.6 MB/s 
     |████████████████████████████████| 561 kB 48.8 MB/s 
     |████████████████████████████████| 80 kB 7.4 MB/s 
     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 131 kB 42.5 MB/s 
     |████████████████████████████████| 131 kB 44.4 MB/s 
     |████████████████████████████████| 130 kB 45.9 MB/s 
     |████████████████████████████████| 130 kB 46.5 MB/s 
     |████████████████████████████████| 13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 11.1 MB 35.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=c9851010a2d2cef0c28311975c57dc64a9fedf814d77ba0e5fc579859daac890
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart


In [5]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse
from keras.preprocessing import image
import uvicorn
import cv2
import numpy as np
from io import BytesIO
from PIL import Image
import nest_asyncio
from pyngrok import ngrok
import tensorflow as tf
from typing import List

app = FastAPI()

ngrok.set_auth_token("2AOQr9YwJxXUwYC8c6mSD6xT8Bb_3ZL8QadVDHhEXnoqmTAme")

MODEL = tf.keras.models.load_model("/content/drive/MyDrive/Image Classification/transfer_model/my_model")

@app.get("/ping")
async def ping():
  return "check"

def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

@app.post("/predict")
async def predict(
    file: UploadFile = File(...)
):
    image = read_file_as_image(await file.read())
    img = cv2.resize(image,(224,224)) 
    img = img.reshape(224,224,3) 

    img_batch = np.expand_dims(img, 0)
  
    predictions = MODEL.predict(img_batch)
    confidence = np.max(predictions[0])
    if confidence >= 0.001 :
      prediction = 'Kertas'
      print(prediction)
    else:
      prediction = 'Buku'
      print(prediction)

    return {
        "name": file.filename,
        'class': prediction,
        'confidence': float(confidence)
    }


In [6]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://7b9d-34-133-45-32.ngrok.io


INFO:     Started server process [61]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2001:448a:4029:18be:3c13:a51:bad3:67ea:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2001:448a:4029:18be:3c13:a51:bad3:67ea:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2001:448a:4029:18be:3c13:a51:bad3:67ea:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:448a:4029:18be:3c13:a51:bad3:67ea:0 - "GET /openapi.json HTTP/1.1" 200 OK
Buku
INFO:     36.65.103.141:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [61]
